In [1]:
%pip install pandas numpy scikit-learn ucimlrepo lime

In [4]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from ucimlrepo import fetch_ucirepo 
import lime
from lime import lime_tabular

import pickle as pkl


MODEL_NAME = 'Wine'

coding = {"Wine": 109}

In [6]:
# fetch dataset 
split_size = 0.3
seed = 1234567

dataset_id = coding[MODEL_NAME]

wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
X = wine.data.features 
y = wine.data.targets 

n_classes = len(y['class'].unique())
n_feats = X.shape[1]
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = split_size, random_state = seed)

normalizer = Normalizer().fit(x_train)
encoder = OneHotEncoder().fit(y_train)

x_train = normalizer.transform(x_train)
x_test = normalizer.transform(x_test)
y_train = encoder.transform(y_train)
y_test  = encoder.transform(y_test)

In [7]:
n_classes, n_feats

(3, 13)

In [8]:
def evaluate_model(model, x, y):
    pred = model.predict(x)
    return accuracy_score(y, pred)

In [9]:
model = MLPClassifier((n_feats, 256, n_classes),
                      activation = 'relu',
                      learning_rate = 'adaptive', 
                      learning_rate_init=0.029, 
                      max_iter = 1000,
                      random_state=3)
model.fit(x_train, y_train)

acc_train = evaluate_model(model, x_train, y_train)
acc_test  = evaluate_model(model, x_test, y_test)

print(acc_train, acc_test)

0.5967741935483871 0.6296296296296297


In [10]:
with open(f'{MODEL_NAME}.pkl', 'wb') as f:
    pkl.dump(model, f)

In [25]:
predicted_labels = model.predict(x_test).toarray().argmax(axis = 1)

In [35]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data = x_train,
    mode = 'classification'
)

idx = 0

exp = explainer.explain_instance(x_test[idx], model.predict_proba, [predicted_labels[idx]] , num_features = n_feats)
local_exp = list(exp.local_exp.values())[0]

local_exp

c:\Users\krish\.conda\envs\torchnew\Lib\site-packages\lime\lime_tabular.py:372: UserWarning: 
                    Prediction probabilties do not sum to 1, and
                    thus does not constitute a probability space.
                    Check that you classifier outputs probabilities
                    (Not log probabilities, or actual class predictions).
                    
  warnings.warn("""


[(9, 0.1848379175010235),
 (3, 0.14315661284391373),
 (6, -0.09554077845091656),
 (8, 0.049502894093396775),
 (4, 0.04783298737703355),
 (10, 0.03850792470559048),
 (11, -0.031199826789448045),
 (2, 0.021128853336642932),
 (7, 0.015179366586021651),
 (5, -0.00976417174586477),
 (1, 0.006906603276165515),
 (12, -0.005951042456801082),
 (0, 0.00441306784448117)]

In [36]:
sorted(local_exp)

[(0, 0.00441306784448117),
 (1, 0.006906603276165515),
 (2, 0.021128853336642932),
 (3, 0.14315661284391373),
 (4, 0.04783298737703355),
 (5, -0.00976417174586477),
 (6, -0.09554077845091656),
 (7, 0.015179366586021651),
 (8, 0.049502894093396775),
 (9, 0.1848379175010235),
 (10, 0.03850792470559048),
 (11, -0.031199826789448045),
 (12, -0.005951042456801082)]

In [13]:
sum([abs(x[1]) for x in exp.local_exp[1]])

KeyError: 1

In [ ]:
# TODO
# add bayesian hyperparameter optimization for finding optimal number of components in gaussian mixture model (https://www.run.ai/guides/hyperparameter-tuning/bayesian-hyperparameter-optimization)
# do sampling and hyperparameter exstimation
# https://www.mdpi.com/2673-2688/4/2/23#:~:text=Post%2Dhoc%20explanation%20methods%20can,of%20underlying%20black%2Dbox%20model.
# https://arxiv.org/pdf/1907.03039.pdf
# https://arxiv.org/pdf/2101.07685.pdf
# https://proceedings.neurips.cc/paper_files/paper/2020/file/24aef8cb3281a2422a59b51659f1ad2e-Paper.pdf
# https://www.analyticsvidhya.com/blog/2018/11/reinforcement-learning-introduction-monte-carlo-learning-openai-gym/
# https://faculty.washington.edu/yenchic/21Sp_stat542/Lec3_Mixture.pdf
# https://machinelearningmastery.com/probabilistic-model-selection-measures/
# https://www.run.ai/guides/hyperparameter-tuning/bayesian-hyperparameter-optimization
# https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html#sklearn.mixture.BayesianGaussianMixture
# https://stats.stackexchange.com/questions/226834/sampling-from-a-mixture-of-two-gamma-distributions/226837#226837

# https://github.com/marcotcr/lime/blob/master/lime/explanation.py
# https://lime-ml.readthedocs.io/en/latest/lime.html#
# https://www.analyticsvidhya.com/blog/2022/07/everything-you-need-to-know-about-lime/
# https://github.com/msetzu/glocalx
# https://drive.google.com/file/d/1gW7V4FyUj9vajrGbIkoa7mbbZn4xEbD6/view


# benchmarks
# https://arxiv.org/pdf/2106.12543.pdf